In [3]:
from torchvision import models
res_model = models.resnet50(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\darthjarjar/.torch\models\resnet50-19c8e357.pth
102502400.0 bytes


NameError: name 'resnet' is not defined

In [6]:
import os
print(os.listdir('data/data_first_25/ApplyEyeMakeup'))


['g01_c01', 'g01_c02', 'g01_c03', 'g01_c04', 'g01_c05', 'g01_c06', 'g02_c01', 'g02_c02', 'g02_c03', 'g02_c04', 'g03_c01', 'g03_c02', 'g03_c03', 'g03_c04', 'g03_c05', 'g03_c06', 'g04_c01', 'g04_c02', 'g04_c03', 'g04_c04', 'g04_c05', 'g04_c06', 'g04_c07', 'g05_c01', 'g05_c02', 'g05_c03', 'g05_c04', 'g05_c05', 'g05_c06', 'g05_c07', 'g06_c01', 'g06_c02', 'g06_c03', 'g06_c04', 'g06_c05', 'g06_c06', 'g06_c07', 'g07_c01', 'g07_c02', 'g07_c03', 'g07_c04', 'g07_c05', 'g07_c06', 'g07_c07', 'g08_c01', 'g08_c02', 'g08_c03', 'g08_c04', 'g08_c05', 'g09_c01', 'g09_c02', 'g09_c03', 'g09_c04', 'g09_c05', 'g09_c06', 'g09_c07', 'g10_c01', 'g10_c02', 'g10_c03', 'g10_c04', 'g10_c05', 'g11_c01', 'g11_c02', 'g11_c03', 'g11_c04', 'g11_c05', 'g12_c01', 'g12_c02', 'g12_c03', 'g12_c04', 'g12_c05', 'g12_c06', 'g13_c01', 'g13_c02', 'g13_c03', 'g13_c04', 'g13_c05', 'g13_c06', 'g14_c01', 'g14_c02', 'g14_c03', 'g14_c04', 'g14_c05', 'g15_c01', 'g15_c02', 'g15_c03', 'g15_c04', 'g15_c05', 'g15_c06', 'g15_c07', 'g16_c01'

In [11]:
#import imageio
from PIL import Image
from torchvision import transforms

test_transform = transforms.Compose(
    [transforms.Resize(256),  # 1. Resize smallest side to 256.
     transforms.CenterCrop(224), # 2. Crop center square of 224x224 pixels.
     transforms.ToTensor(), # 3. Convert to pytorch tensor.
     transforms.Normalize(mean = [0.485, 0.456, 0.406],  # normalize.
                          std = [0.229, 0.224, 0.225])
    ])

img_pil = Image.open('data/data_first_25/ApplyEyeMakeup/g01_c01/v_ApplyEyeMakeup_g01_c01_frame0.jpg')

# 1. Forward propagate the image through the CNN.
# Unsqueeze adds a dummy batch dimension needed to pass through the model.
input_img =  test_transform(img_pil).unsqueeze(0)

print(res_model(input_img))

tensor([[-1.0339e+00, -2.8034e-01, -5.5877e-01, -1.0633e+00, -5.9821e-01,
         -1.1849e-01, -7.2454e-01,  4.7234e-01,  1.6870e-01, -7.7246e-01,
         -8.7720e-01, -9.8966e-01, -3.8633e-01, -9.5370e-01, -8.5343e-01,
         -4.4967e-01, -6.4921e-01, -1.9972e-01, -1.6484e-01, -4.8417e-01,
         -1.0847e+00, -7.8658e-01, -1.3034e+00, -1.1527e-01, -1.0191e+00,
         -7.0221e-01, -8.2755e-01, -6.5090e-01, -5.0219e-01, -4.1313e-01,
         -7.6655e-01, -8.9262e-01, -6.4929e-01, -4.9522e-01, -2.0277e-01,
         -5.8173e-01,  5.8565e-01, -6.6301e-01, -3.7567e-01,  1.5913e-01,
         -5.8464e-01, -6.5548e-01, -6.9372e-01, -2.6466e-01, -2.6957e-01,
         -2.1896e-01, -5.5571e-01, -4.5805e-01, -1.1228e+00, -7.5635e-01,
         -2.6246e-01,  3.9869e-01, -2.5952e-01, -3.1649e-01, -1.7936e-01,
         -9.0641e-01, -4.0234e-01, -9.8930e-01, -2.5644e-01,  1.6537e-02,
          7.0540e-01,  2.3544e-01, -9.1689e-03,  2.5921e-02, -5.4208e-01,
         -1.6710e-01, -3.3157e-01, -1.